# **퇴근시간 버스 승차인원 예측**( Baselin, 초급용 코드 )

 1. 배경
 ```
 제주도 내 주민등록 인구는 19년 11월 기준 69만명으로, 연평균 4%대로 성장했다.
 외국인과 관광객까지 고려하면 전체 상주인구는 90만명을 넘을 것으로 추정된다.
 제주도민 증가와 외국인의 증가로 현재 제주도 일부 지역의 교통체증은 서울보다 심각하다.
  따라서, 제주테크노파크는 교통난 심화를 해결하기 위해 데이터 분석대회를 개최한다.
 ```
 2. 데이터
  -  train, test, bus_bts 공통 사항
     ```
    해당 데이터에는 버스카드를 통해 결제한 경우에 대한 정류소 승,하차 데이터로
    모든 승차정보는 기록되어 있지만, 
    버스 하차시 카드 찍히지 않는 경우는 비어있는 상태.
    따라서, 승하차 인원수에 차이가 있을 수 있다.
     ```
  
  - train,test 공통 사항
     ```
     해당 버스정류장에 대한 각각의 위도, 경도가 제공된 상태로 같은 정류장 이름이나 
     위도와 경도가 서로 다른 경우가 존재한다.
     해당 경우, 같은 정류장 이름을 갖는 길 건너편 정류장에 해당.
     ```


In [73]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import warnings 
import geopy.distance
warnings.filterwarnings('ignore')

In [96]:
train = pd.read_csv('/content/drive/MyDrive/dacon/승차인원 예측/train.csv')
test = pd.read_csv('/content/drive/MyDrive/dacon/승차인원 예측/test.csv')

In [ ]:
train.head()

In [ ]:
test.head()

## 1 .date 변수 변환


In [99]:
train['date'] = pd.to_datetime(train['date'])
train['weekday'] = train['date'].dt.weekday
train = pd.get_dummies(train,columns=['weekday']) # 요일을 0~6까지 숫자로 표기(6:sunday) // pd.get_dummies(data, columns) 일 경우 기존 df에 one hot encoding 수행

In [100]:
test['date']=pd.to_datetime(test['date'])
test['weekday']=test['date'].dt.weekday
test=pd.get_dummies(test,columns=['weekday'])

In [ ]:
test.head()

## 2  .in_out 컬럼 변환


In [102]:
train['in_out'].value_counts()
#=> '시외' 매우 적다.

train['in_out']=train['in_out'].map({'시내':0, '시외':1})
test['in_out'] = test['in_out'].map({'시내':0, '시외':1})


## 3  . 좌표 데이터를 이용한 Feature Engineering
 > 제주 인구는 서귀포시와 제주시에 몰려있다 볼 수 있다. 그러므로 해당 지역의 거리를 각각 Feature로 추가.

  * dis_jejusi : 버스정류장과 제주시 거리
  * dis_seoquiop : 버스정류장과 서귀포시 거리
  * 제주시의 위/경도 : 33.500770, 126,522761
  * 서귀포시의 위/경도 : 33.25429, 126.558217
    * (2개 좌표간 거리를 구하는 geopy 패키지 사용)

In [103]:
!pip install geopy

In [104]:
import geopy.distance
coords_jejusi = (33.500770, 126.522761)    # 제주시 위/경도
coords_seoquipo = (33.259429, 126.558217)  # 서귀포시 위/경도

# train['dis_jejusi'] 
train['dis_jejusi']= [geopy.distance.vincenty((train['latitude'].iloc[i],train['longitude'].iloc[i]), coords_jejusi).km for i in range(len(train))]
train['dis_seoquipo']=[geopy.distance.vincenty((train['latitude'].iloc[i],train['longitude'].iloc[i]), coords_seoquipo).km for i in range(len(train))]

In [105]:
train['dis_jejusi'].corr(train['18~20_ride'])

-0.12380548460763058

In [106]:
train['dis_seoquipo'].corr(train['18~20_ride'])

0.022087013666358277

```
제주시와 거리가 멀면 승차인원 감소,
서귀포시의 거리와는 뚜렷한 상관관계 볼 수 없다.
```

In [107]:
test['dis_jejusi']= [geopy.distance.vincenty((train['latitude'].iloc[i],test['longitude'].iloc[i]), coords_jejusi).km for i in range(len(test))]
test['dis_seoquipo']=[geopy.distance.vincenty((train['latitude'].iloc[i],test['longitude'].iloc[i]), coords_seoquipo).km for i in range(len(test))]

## 4  . RandomForest 활용한 모델링
sklearn 내부의 RandomForest 함수를 이용해 모델링을 진행한다.
머신러닝 학습에 활용할 input 변수와 target 변수를 정의한다

In [38]:
input_var = train.columns.difference(['18~20_ride'])
target = ['18~20_ride']

In [ ]:
input_var=['in_out','latitude','longitude','6~7_ride', '7~8_ride', '8~9_ride', '9~10_ride',
       '10~11_ride', '11~12_ride', '6~7_takeoff', '7~8_takeoff', '8~9_takeoff',
       '9~10_takeoff', '10~11_takeoff', '11~12_takeoff','weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4',
       'weekday_5', 'weekday_6', 'dis_jejusi', 'dis_seoquipo']
target=['18~20_ride']

In [ ]:
X_train=train[input_var]
y_train=train[target]

X_test=test[input_var]

In [2]:
# 랜덤포레스트 모델 정의
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state = 1217)

In [ ]:
rf.fit(X_train, y_train)

In [ ]:
test['18~20_ride'] =rf.predict(X_test)
test[['id','18~20_ride']].to_csv('base_line.csv', index=False)

# part 2) train, test 병합 및 전처리

In [4]:
train = pd.read_csv('/content/drive/MyDrive/dacon/승차인원 예측/train.csv')
test = pd.read_csv('/content/drive/MyDrive/dacon/승차인원 예측/test.csv')

## 2-1  . 데이터 병합

In [5]:
train.set_index('id',inplace = True)
test.set_index('id',inplace=True)

In [6]:
train_index = train.index
test_index = test.index

In [7]:
y_train = train.pop('18~20_ride')

In [8]:
all_df=train.append(test)

## 2-2. 전처리

In [9]:
all_df['date'] = pd.to_datetime(all_df['date'])
all_df['weekday']=all_df['date'].dt.weekday
all_df=pd.get_dummies(all_df, columns=['weekday'])

In [10]:
all_df['in_out'].value_counts()
#=> '시외' 매우 적다.
all_df['in_out']=all_df['in_out'].map({'시내':0, '시외':1})

In [11]:
coords_jejusi = (33.500770, 126.522761) #제주시의 위도 경도
coords_seoquipo = (33.259429, 126.558217) #서귀포시의 위도 경도

all_df['dis_jejusi'] = [geopy.distance.vincenty((all_df['latitude'].iloc[i],all_df['longitude'].iloc[i]), coords_jejusi).km for i in range(len(all_df))]
all_df['dis_seoquipo'] = [geopy.distance.vincenty((all_df['latitude'].iloc[i],all_df['longitude'].iloc[i]), coords_seoquipo).km for i in range(len(all_df))]


In [12]:
all_df['dis_jejusi'].corr(y_train)

-0.12380548460763058

In [52]:
all_df['dis_seoquipo'].corr(y_train)

0.022087013666358277

## 2-3. 데이터 분리

In [13]:
# 데이터 분리
X_train = all_df[:len(train_index)]
X_test = all_df[len(train_index):]
X_test.reset_index(inplace=True, drop=True)


In [ ]:
input_var=['in_out','latitude','longitude','6~7_ride', '7~8_ride', '8~9_ride', '9~10_ride',
       '10~11_ride', '11~12_ride', '6~7_takeoff', '7~8_takeoff', '8~9_takeoff',
       '9~10_takeoff', '10~11_takeoff', '11~12_takeoff','weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4',
       'weekday_5', 'weekday_6', 'dis_jejusi', 'dis_seoquipo']
target=['18~20_ride']

In [20]:
X_train = X_train[input_var]
X_test = X_test[input_var]

## 2-4. Modeling

In [14]:
# 랜덤포레스트 모델 정의
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=1217)

In [21]:
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=1217, verbose=0, warm_start=False)

In [22]:
test['18~20_ride'] =rf.predict(X_test)
test[['id','18~20_ride']].to_csv('base_line.csv', index=False)

KeyError: ignored